In [ ]:
import numpy as np

In [ ]:
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit, RepeatedStratifiedKFold

def _validate(clf, clf_name, 
                X_test, y_test,
                metric_name, metric_params):
    if clf_name.startswith('ml_'):
        # Make the predictions
        y_pred = clf.predict_proba(X_test)[:,1]
    elif clf_name.startswith('cs_'):
        # apply consensus
        y_pred = clf(pd.DataFrame(X_test))
    else:
        print(clf_name, 'not found. Ommited.')
        return
    
    # Make the evaluation
    metric = PlotMetric(y_test, {'': y_pred},
                decreasing=False)\
                .format_metric_results(
                    rounded=5,
                    #metric_name=metric_name,
        # metric_params already includes metric_name
                    **metric_params)

    return metric.values[0][0]


def _train_cfl(clf, X_train, y_train):
    # Fit the estimator
    clf.fit(X_train, y_train)
    return clf
    

def _do_replicates(splits, 
                   estimators, X, y,
                   metrics):
    results={}
    # Machine Learning Classifiers
    for clf_name, clf in estimators.items():
        folds = []
        for i, (train, test) in enumerate(splits):
            if clf_name.startswith('ml_'):
                # Fit the ml classifier once per fold
                cfl = _train_cfl(clf, X[train], y[train])
            
            # for each metric
#             metric_results = {}
            for metric_name, metric_params in metrics.items():
            
                metric = _validate(
                    clf, clf_name, 
                    X[test], y[test],
                    metric_name, metric_params
                )
                # Append the results
                folds.append(metric)

        # Add to the results dictonary 
        results[clf_name] = folds

    return results

def _format_results_to_df(metrics, results, n):
        # Format into a dataframe
    # Create the metric names and repeat them 
    n_metrics = len(metrics)
    index_names = [*metrics.keys()]*n
    
    # convert to a dataframe
    df_res = pd.DataFrame(
        results, 
        index= pd.MultiIndex.from_tuples(
            zip(index_names,
                np.repeat(range(n), n_metrics))
        ))
    df_res = df_res.sort_index()
    
    return df_res


@cached()
def k_cross_validation(
          estimators, X, y,
          metrics,
          n_splits=5, 
          random_state=None, 
          shuffle=True):
    # Compute the Stratified K folds
    cv = StratifiedKFold(n_splits=n_splits, 
                         random_state=random_state,
                         shuffle=shuffle)
    splits = [*cv.split(X, y)]
    
    results = _do_replicates(splits, estimators, X, y, 
                             metrics)
    
    df_res = _format_results_to_df(metrics, results, n=n_splits)
    
    return df_res 


@cached()
def n_hold_out_validation(
          estimators, X, y,
          metrics,
          n_reps=5, test_size=0.25,
          random_state=None):
    # Compute the Stratified K folds
    cv = StratifiedShuffleSplit(
                        n_splits=n_reps, 
                        test_size=test_size,
                        random_state=random_state)
    splits = [*cv.split(X, y)]
    
    results = _do_replicates(splits, estimators, X, y,
          metrics)
    
    df_res = _format_results_to_df(metrics, results, n=n_reps)
    
    return df_res 


@cached()
def nk_rep_cross_validation(
          estimators, X, y,
          metrics,
          n_splits=2, 
          n_repeats=5,
          random_state=None, 
          shuffle=True):
    # Compute the Stratified K folds
    cv = RepeatedStratifiedKFold(
                         n_splits=n_splits,
                         n_repeats=n_repeats,
                         random_state=random_state)
    splits = [*cv.split(X, y)]
    
    results = _do_replicates(splits, estimators, X, y, 
                             metrics)
    
    df_res = _format_results_to_df(metrics, results, n=n_splits*n_repeats)
    
    return df_res 

In [ ]:
from sklearn.metrics import auc
from sklearn.metrics import plot_roc_curve
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns

@cached()
def plot_roc_cv(classifier, X, y, random_state=None,
               n_folds=5, ax=None, name=''):
    sns.set(style='whitegrid', font_scale=1.2)
    
    cv = StratifiedKFold(n_splits=n_folds, random_state=random_state, shuffle=True)

    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)
    
    if ax == None:
        fig, ax = plt.subplots()
    for i, (train, test) in enumerate(cv.split(X, y)):
        classifier.fit(X[train], y[train])
        viz = plot_roc_curve(classifier, X[test], y[test],
                             name='ROC fold {}'.format(i),
                             alpha=0.3, lw=1, ax=ax)
        interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(viz.roc_auc)

    ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
            label='Random', alpha=.8)

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(mean_fpr, mean_tpr, color='b',
            label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
            lw=2, alpha=.8)

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                    label=r'$\pm$ 1 std. dev.')

    ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
    ax.set_title(label=f"{n_folds}-fold CV ROC curve: {name}", fontsize=16, fontweight='bold')
    ax.legend(loc="lower right")
    return ax

In [ ]:
from scipy.stats import shapiro, bartlett, levene

def norm_test(x, alpha=0.05):
    s, p = shapiro(x)
    result = 'rejected' if p < alpha else 'accepted'
    print(f'The H0 is {result} => (W={round(s, 3)}, p={round(p, 3)})')
    
def homovar_test(x, y, alpha=0.05):
    s, p = bartlett(x, y)
    result = 'rejected' if p < alpha else 'accepted'
    print(f'The H0 is {result} => (W={round(s, 3)}, p={round(p, 3)})')
    
def multi_norm_test(df, metric='roc_auc', alpha=0.05):
    res = df.loc[metric].apply(shapiro, axis=0)
    return pd.DataFrame([0 if i[1] < alpha else 1 for i in res], 
             index=df.columns, columns=['Normality']).T

def multi_homovar_test(df, metric='roc_auc', alpha=0.05, as_df=True):
    res = bartlett(*df.loc[metric].values)
    if as_df:
        res = pd.DataFrame(res, columns=['Bartlett'], 
                           index=['statistic', 'p']).T
    return res

In [ ]:
# Memory memoization
from joblib import Memory
location = './cachedir'
memory = Memory(location, verbose=0)

n_hold_out_validation = memory.cache(n_hold_out_validation) 

k_cross_validation = memory.cache(k_cross_validation) 

nk_rep_cross_validation = memory.cache(nk_rep_cross_validation) 